In [1]:
import pandas as pd
import json


original = pd.read_csv('data.csv')
original = original.drop(['都道府県名ローマ字', '市区町村名ローマ字', '大字町丁目コード', '大字町丁目名', '緯度', '経度'], axis=1)
original = original.rename(columns={'都道府県コード':  'prefecture_code', 
                                    '都道府県名': ' prefecture_name',
                                    '都道府県名カナ': 'prefecture_name_ruby', 
                                    '市区町村コード': 'city_code',
                                    '市区町村名': 'city_name',
                                    '市区町村名カナ': 'city_name_ruby',})


prefecture = original.iloc[:,:3]
counter_prefecture = prefecture.iloc[0, 0]
drop_prefecture = []
for i in range(1, len(prefecture)):
    if prefecture.iloc[i, 0] == counter_prefecture:
        drop_prefecture.append(i)
    else:
        counter_prefecture = prefecture.iloc[i, 0]
        continue
prefecture = prefecture.drop(drop_prefecture)
prefecture.reset_index(inplace=True, drop=True)
print(prefecture)

city = original.drop(columns=original.columns[[1, 2]])
counter_city = city.iloc[0,1]
drop_city = []
for i in range(1, len(city) - 1):
    if city.iloc[i, 1] == counter_city:
        drop_city.append(i)
    else:
        counter_city = city.iloc[i, 1]
        continue
city =  city.drop(drop_city)
city.reset_index(inplace=True, drop=True)
print(city)

    prefecture_code  prefecture_name prefecture_name_ruby
0                 1              北海道               ホッカイドウ
1                 2              青森県               アオモリケン
2                 3              岩手県                イワテケン
3                 4              宮城県                ミヤギケン
4                 5              秋田県                アキタケン
5                 6              山形県               ヤマガタケン
6                 7              福島県               フクシマケン
7                 8              茨城県               イバラキケン
8                 9              栃木県                トチギケン
9                10              群馬県                グンマケン
10               11              埼玉県               サイタマケン
11               12              千葉県                 チバケン
12               13              東京都               トウキョウト
13               14             神奈川県               カナガワケン
14               15              新潟県               ニイガタケン
15               16              富山県                トヤマケン
16            

In [14]:
import json
from pathlib import Path


def json_make(path: Path, obj: dict) -> None:
    ls = None
    with open(path, 'r+') as f:
        ls = f.readlines()
        if ls == []:
            ls.append('[\n')
        if ls[-1] == ']':
            ls[-1] = ','
        ls.insert(len(ls), f'{json.dumps(obj, indent=4 ,ensure_ascii=False)}')
        ls.insert(len(ls), '\n]')

    with open(path, 'w') as f:
        f.writelines(ls)


for i in range(len(prefecture)):
    base_dict_prefecture = {
        "pk" : "",
        "model" : "api.MasterPrefecture",
        "fields" : {
            "prefecture_name" : "",
            "prefecture_name_ruby" : ""
        }
    }
    base_dict_prefecture["pk"] = str(prefecture.iloc[i, 0])
    base_dict_prefecture["fields"]["prefecture_name"] = str(prefecture.iloc[i, 1])
    base_dict_prefecture["fields"]["prefecture_name_ruby"] = str(prefecture.iloc[i, 2])
    json_make("./src/api/fixtures/prefecture.json", base_dict_prefecture)

for i in range(len(city)):
    base_dict_city = {
        "pk" : "",
        "model" : "api.MasterCity",
        "fields" : {
            "prefecture_code" : "",
            "city_name" : "",
            "city_name_ruby" : ""
        }
    }
    base_dict_city["pk"] = str(city.iloc[i, 1])
    base_dict_city["fields"]["prefecture_code"] = str(city.iloc[i, 0])
    base_dict_city["fields"]["city_name"] = str(city.iloc[i, 2])
    base_dict_city["fields"]["city_name_ruby"] = str(city.iloc[i, 3])
    json_make("./src/api/fixtures/city.json", base_dict_city)
